## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-30-10-53-00 +0000,wsj,Tsunami Waves Reach U.S. After Powerful Russia...,https://www.wsj.com/world/tsunami-watch-warnin...
1,2025-07-30-10-50-23 +0000,bbc,Lammy hopes plan to recognise Palestinian stat...,https://www.bbc.com/news/articles/c23p8gl05r2o...
2,2025-07-30-10-48-00 +0000,wsj,Fed Meeting Today: S&P 500 Futures Edge Up Wit...,https://www.wsj.com/livecoverage/fed-meeting-i...
3,2025-07-30-10-47-24 +0000,nyt,In Conversation with Brazil’s President Lula,https://www.nytimes.com/video/world/americas/1...
4,2025-07-30-10-46-52 +0000,nyt,Live Updates: Tsunami Waves Reach California a...,https://www.nytimes.com/live/2025/07/29/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
48,trump,63
0,tsunami,21
420,new,17
13,palestinian,17
219,gaza,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
161,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...,123
91,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...,123
93,2025-07-30-01-30-00 +0000,wsj,The Senate confirmed former Trump defense atto...,https://www.wsj.com/politics/policy/trumps-for...,118
252,2025-07-29-17-49-06 +0000,nypost,"Moscow taunts Trump by comparing him to Biden,...",https://nypost.com/2025/07/29/world-news/mosco...,113
170,2025-07-29-21-42-41 +0000,nypost,Top Zelensky official praises Trump for giving...,https://nypost.com/2025/07/29/us-news/top-zele...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
161,123,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...
294,75,2025-07-29-15-30-51 +0000,nypost,Union Pacific strikes $85B deal to buy rival N...,https://nypost.com/2025/07/29/business/union-p...
89,68,2025-07-30-01-49-26 +0000,nypost,Reno casino victim ID’d as Calif. dad who advo...,https://nypost.com/2025/07/29/us-news/reno-cas...
91,60,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...
61,59,2025-07-30-04-38-58 +0000,bbc,Watch: Major earthquake off Russia triggers wi...,https://www.bbc.com/news/videos/cwy01l5kgxno?a...
28,56,2025-07-30-09-30-00 +0000,wsj,The shake-up for charitable giving inside the ...,https://www.wsj.com/politics/policy/trump-tax-...
234,52,2025-07-29-18-32-06 +0000,nypost,UK will recognize Palestinian state unless Isr...,https://nypost.com/2025/07/29/world-news/uk-wi...
122,45,2025-07-30-00-06-03 +0000,nypost,NYC skyscraper that turned into tower of terro...,https://nypost.com/2025/07/29/us-news/nyc-skys...
299,41,2025-07-29-15-07-33 +0000,latimes,Shane Tamura played high school football in L....,https://www.latimes.com/california/story/2025-...
200,40,2025-07-29-20-46-00 +0000,wsj,The Dow Jones Industrial Average fell more tha...,https://www.wsj.com/finance/stocks/s-p-500-bre...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
